In [1]:
import itertools
import os
import sys
from itertools import combinations_with_replacement
from math import e

import numpy as np
import pandas as pd
from numba import njit
from plotly.subplots import make_subplots
from vectorbt.indicators import nb

from lib.utils import file_to_data_frame, LR, ExtendedPortfolio, plot_series_vs_scatters, dropnaninf, get_best_index

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import vectorbt as vbt


In [2]:
file = "/Users/pilo/development/itba/pf/Binance_Minute_OHLC_CSVs/mediums/Binance_ADAUSDT_minute_10000.csv"
_, ohlcv = file_to_data_frame(file)
ohlcv. head()

,Open,High,Low,Close,Volume ADA,Volume,Tradecount
date,,,,,,,
2021-03-12 01:29:00,1.12163,1.12331,1.12163,1.12238,147940.6,166105.769425,153
2021-03-12 01:30:00,1.12238,1.12247,1.12100,1.12190,100464.7,112694.400152,182
2021-03-12 01:31:00,1.12183,1.12268,1.12183,1.12265,98108.4,110099.944426,136
2021-03-12 01:32:00,1.12264,1.12307,1.12224,1.12244,183630.5,206139.570488,184
2021-03-12 01:33:00,1.12255,1.12279,1.12143,1.12178,89694.8,100645.251713,174


In [3]:
close = ohlcv["Close"]
volume = ohlcv["Volume"]
lr_ind = LR.run(close)
print(lr_ind.lr.shape)

(9999,)


In [4]:
lag = list(range(7,15))
ma_ind = vbt.MA.run(lr_ind.lr, lag, short_name="lr_ma")
mstd_ind = vbt.MSTD.run(lr_ind.lr, lag)

In [5]:
@njit
def combination_nb(ma, std, thld):
    return ma + thld * std

LR_MULTIPLIER = vbt.IndicatorFactory(
    input_names=['ma', 'std'],
    param_names=['thld'],
    output_names=['mu']
).from_apply_func(combination_nb, use_ray=True)
#u_t = 0.08 # np.linspace(0,2, 10, endpoint=True)
d_t = -0.08 # np.linspace(0,2, 10, endpoint=True)
# ups = LR_MULTIPLIER.run(ma_ind.ma, mstd_ind.mstd, thld=u_t, short_name="ups_mu")
pitfalls = LR_MULTIPLIER.run(ma_ind.ma, mstd_ind.mstd, thld=d_t, short_name="pitfalls_mu")
pitfalls.mu.head()

pitfalls_mu_thld    -0.08                            
mstd_window            7   8   9   10  11  12  13  14
date                                                 
2021-03-12 01:29:00   NaN NaN NaN NaN NaN NaN NaN NaN
2021-03-12 01:30:00   NaN NaN NaN NaN NaN NaN NaN NaN
2021-03-12 01:31:00   NaN NaN NaN NaN NaN NaN NaN NaN
2021-03-12 01:32:00   NaN NaN NaN NaN NaN NaN NaN NaN
2021-03-12 01:33:00   NaN NaN NaN NaN NaN NaN NaN NaN

In [6]:
lr_entries = lr_ind.lr_below(pitfalls.mu)
#lr_exits = lr_ind.lr_above(ups.mu)
lr_entries.head()

pitfalls_mu_thld     -0.08                                                 
mstd_window             7      8      9      10     11     12     13     14
date                                                                       
2021-03-12 01:29:00  False  False  False  False  False  False  False  False
2021-03-12 01:30:00  False  False  False  False  False  False  False  False
2021-03-12 01:31:00  False  False  False  False  False  False  False  False
2021-03-12 01:32:00  False  False  False  False  False  False  False  False
2021-03-12 01:33:00  False  False  False  False  False  False  False  False

In [7]:
vol_ma = vbt.MA.run(volume, lag)
vol_ma.ma.head()

ma_window,7,8,9,10,11,12,13,14
date,,,,,,,,
2021-03-12 01:29:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-12 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-12 01:31:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-12 01:32:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-12 01:33:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
@njit
def multiplier_nb(values, m):
    return m * values

MULTIPLIER = vbt.IndicatorFactory(
    input_names=['values'],
    param_names=['m'],
    output_names=['mu']
).from_apply_func(multiplier_nb)

In [9]:
# Volume part:
vol_threshold = 1.5 # np.linspace(0.995839, 2, 20)
vol_multiplier = MULTIPLIER.run(vol_ma.ma, vol_threshold, short_name="vol_mu")
# TODO filtrar los que no generan entries
vol_entries = vol_multiplier.mu_below(volume)
print("sum", vol_entries.sum(axis=0))
print("shape", volume.shape)
vol_entries.head()

sum vol_mu_m  ma_window
1.5       7            1361
          8            1393
          9            1391
          10           1438
          11           1440
          12           1469
          13           1483
          14           1482
dtype: int64
shape (9999,)


vol_mu_m               1.5                                                 
ma_window               7      8      9      10     11     12     13     14
date                                                                       
2021-03-12 01:29:00  False  False  False  False  False  False  False  False
2021-03-12 01:30:00  False  False  False  False  False  False  False  False
2021-03-12 01:31:00  False  False  False  False  False  False  False  False
2021-03-12 01:32:00  False  False  False  False  False  False  False  False
2021-03-12 01:33:00  False  False  False  False  False  False  False  False

In [10]:
lr_entries.head()

pitfalls_mu_thld     -0.08                                                 
mstd_window             7      8      9      10     11     12     13     14
date                                                                       
2021-03-12 01:29:00  False  False  False  False  False  False  False  False
2021-03-12 01:30:00  False  False  False  False  False  False  False  False
2021-03-12 01:31:00  False  False  False  False  False  False  False  False
2021-03-12 01:32:00  False  False  False  False  False  False  False  False
2021-03-12 01:33:00  False  False  False  False  False  False  False  False

In [11]:
lr_entries.columns = lr_entries.columns.rename("lag", level=-1)
vol_entries.columns = vol_entries.columns.rename("lag", level=-1)
vol_entries

vol_mu_m               1.5                                                 
lag                     7      8      9      10     11     12     13     14
date                                                                       
2021-03-12 01:29:00  False  False  False  False  False  False  False  False
2021-03-12 01:30:00  False  False  False  False  False  False  False  False
2021-03-12 01:31:00  False  False  False  False  False  False  False  False
2021-03-12 01:32:00  False  False  False  False  False  False  False  False
2021-03-12 01:33:00  False  False  False  False  False  False  False  False
...                    ...    ...    ...    ...    ...    ...    ...    ...
2021-03-19 00:03:00   True   True   True   True   True   True   True   True
2021-03-19 00:04:00  False  False  False  False  False   True   True   True
2021-03-19 00:05:00  False  False  False  False  False  False  False  False
2021-03-19 00:06:00  False  False  False  False  False  False  False  False
2021-03-19 00:07:00  False  False  False  False  False  False  False  False

[9999 rows x 8 columns]

In [12]:
final_entries = lr_entries & vol_entries
final_entries

lag,7,8,9,10,11,12,13,14
pitfalls_mu_thld,-0.08,-0.08,-0.08,-0.08,-0.08,-0.08,-0.08,-0.08
vol_mu_m,1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5
date,,,,,,,,
2021-03-12 01:29:00,False,False,False,False,False,False,False,False
2021-03-12 01:30:00,False,False,False,False,False,False,False,False
2021-03-12 01:31:00,False,False,False,False,False,False,False,False
2021-03-12 01:32:00,False,False,False,False,False,False,False,False
2021-03-12 01:33:00,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
2021-03-19 00:03:00,False,False,False,False,False,False,False,False


In [13]:
@njit
def double_multiplier_nb(values, x, y):
    return values*x, values*y

MULTIPLIER = vbt.IndicatorFactory(
    input_names=['values'],
    param_names=['x', 'y'],
    output_names=['x_mu','y_mu']
).from_apply_func(double_multiplier_nb)
x = np.linspace(0,2,5, endpoint=True)
y = -np.linspace(0,2,5, endpoint=True)
tp_sl = MULTIPLIER.run(mstd_ind.mstd, x, y, param_product=True, short_name="tp_sl")

In [14]:
tp_exits = lr_ind.lr_above(tp_sl.x_mu)
sl_exits = lr_ind.lr_below(tp_sl.y_mu)
final_exits = tp_exits | sl_exits
final_exits.columns = final_exits.columns.rename("lag", level=-1)
final_exits

tp_sl_x                0.0                                                   \
tp_sl_y               -0.0                                                    
lag                     7      8      9      10     11     12     13     14   
date                                                                          
2021-03-12 01:29:00  False  False  False  False  False  False  False  False   
2021-03-12 01:30:00  False  False  False  False  False  False  False  False   
2021-03-12 01:31:00  False  False  False  False  False  False  False  False   
2021-03-12 01:32:00  False  False  False  False  False  False  False  False   
2021-03-12 01:33:00  False  False  False  False  False  False  False  False   
...                    ...    ...    ...    ...    ...    ...    ...    ...   
2021-03-19 00:03:00   True   True   True   True   True   True   True   True   
2021-03-19 00:04:00   True   True   True   True   True   True   True   True   
2021-03-19 00:05:00   True   True   True   True   True   True   True   True   
2021-03-19 00:06:00   True   True   True   True   True   True   True   True   
2021-03-19 00:07:00   True   True   True   True   True   True   True   True   

tp_sl_x                            ...    2.0                              \
tp_sl_y               -0.5         ...   -1.5          -2.0                 
lag                     7      8   ...     13     14     7      8      9    
date                               ...                                      
2021-03-12 01:29:00  False  False  ...  False  False  False  False  False   
2021-03-12 01:30:00  False  False  ...  False  False  False  False  False   
2021-03-12 01:31:00  False  False  ...  False  False  False  False  False   
2021-03-12 01:32:00  False  False  ...  False  False  False  False  False   
2021-03-12 01:33:00  False  False  ...  False  False  False  False  False   
...                    ...    ...  ...    ...    ...    ...    ...    ...   
2021-03-19 00:03:00  False  False  ...  False  False  False  False  False   
2021-03-19 00:04:00   True   True  ...  False   True  False  False  False   
2021-03-19 00:05:00  False  False  ...  False  False  False  False  False   
2021-03-19 00:06:00   True   True  ...  False  False  False  False  False   
2021-03-19 00:07:00  False  False  ...  False  False  False  False  False   

tp_sl_x                                                 
tp_sl_y                                                 
lag                     10     11     12     13     14  
date                                                    
2021-03-12 01:29:00  False  False  False  False  False  
2021-03-12 01:30:00  False  False  False  False  False  
2021-03-12 01:31:00  False  False  False  False  False  
2021-03-12 01:32:00  False  False  False  False  False  
2021-03-12 01:33:00  False  False  False  False  False  
...                    ...    ...    ...    ...    ...  
2021-03-19 00:03:00  False  False  False  False  False  
2021-03-19 00:04:00  False  False  False  False   True  
2021-03-19 00:05:00  False  False  False  False  False  
2021-03-19 00:06:00  False  False  False  False  False  
2021-03-19 00:07:00  False  False  False  False  False  

[9999 rows x 200 columns]

In [15]:
portfolio_kwargs = dict(
    direction='longonly',
    freq='m',
)
port = ExtendedPortfolio.from_signals(close, entries=final_entries, exits=final_exits, **portfolio_kwargs, max_logs=0)

In [26]:
elr = port.expected_log_returns()
top_elr = elr.nlargest(25, keep="all")
top_elr

pitfalls_mu_thld  vol_mu_m  tp_sl_x  tp_sl_y  lag
-0.08             1.5       0.0      -0.0     10     0.001647
                                              11     0.001647
                                              12     0.001647
                                              13     0.001647
                                              14     0.001647
                            2.0      -2.0     9      0.001333
                                     -1.5     9      0.000993
                                     -2.0     8      0.000977
                                     -1.0     13     0.000894
                            1.5      -1.0     12     0.000894
                                              13     0.000844
                            1.0      -1.0     10     0.000767
                            1.5      -1.0     10     0.000755
                            2.0      -1.0     12     0.000755
                                     -2.0     11     0.000753
                    

In [27]:
sharpe = dropnaninf(port.sharpe_ratio())
top_sharpe = sharpe.nlargest(25, keep="all")
top_sharpe

pitfalls_mu_thld  vol_mu_m  tp_sl_x  tp_sl_y  lag
-0.08             1.5       0.0      -2.0     9      26.370149
                                              10     25.805595
                                              11     25.699094
                                              14     25.386885
                                     -1.5     10     25.290469
                                     -2.0     12     24.952630
                            0.5      -2.0     9      23.708706
                            0.0      -1.5     13     23.422687
                                              11     23.328026
                                     -2.0     13     23.283684
                                     -1.5     9      23.217979
                            0.5      -2.0     14     23.002972
                            0.0      -1.5     14     22.036082
                                              12     21.236375
                                     -2.0     8      21.051167
     

In [34]:
intsect= top_elr.index.intersection(top_sharpe.index)
intsect

MultiIndex([(-0.08, 1.5, 1.0, -2.0, 12),
            (-0.08, 1.5, 1.0, -2.0, 13),
            (-0.08, 1.5, 1.0, -2.0, 14),
            (-0.08, 1.5, 0.5, -2.0,  9),
            (-0.08, 1.5, 0.0, -2.0,  9)],
           names=['pitfalls_mu_thld', 'vol_mu_m', 'tp_sl_x', 'tp_sl_y', 'lag'])

In [42]:
port.stats(column=top_sharpe.index[0])
#port.plot(column=top_elr.index[0]).show()

Start                           2021-03-12 01:29:00
End                             2021-03-19 00:07:00
Duration                            6 days 22:39:00
Init. Cash                                    100.0
Total Profit                              27.283952
Total Return [%]                          27.283952
Benchmark Return [%]                       8.340313
Position Coverage [%]                      7.030703
Max. Drawdown [%]                          2.945813
Avg. Drawdown [%]                          0.430499
Max. Drawdown Duration              1 days 00:33:00
Avg. Drawdown Duration              0 days 01:51:00
Num. Trades                                     391
Win Rate [%]                              72.890026
Best Trade [%]                              1.49211
Worst Trade [%]                           -0.893109
Avg. Trade [%]                             0.062062
Max. Trade Duration                 0 days 00:07:00
Avg. Trade Duration       0 days 00:01:47.877237851
Expectancy  

In [20]:
print("trades", len(port.trades))

trades 39916


In [21]:
final_entries_plot = (final_entries.where(final_entries == True, np.nan)).vbt.scatterplot()
lr_plot = plot_series_vs_scatters([lr_ind.lr, pitfalls.mu], [lr_entries, lr_exits])
vol_plot = plot_series_vs_scatters([vol_multiplier.mu, volume], [vol_entries])

ValueError: Must specify axis=0 or 1

In [ ]:
def add_all_subplots(fig, row, col, list):
    for a in list:
        fig.add_trace(a, row=row, col=col)

In [ ]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
                    vertical_spacing=0.02)
add_all_subplots(fig, 1, 1, lr_plot.data)
add_all_subplots(fig, 2, 1, vol_plot.data)
add_all_subplots(fig, 3, 1, final_entries_plot.data)
fig.update_layout(height=700, legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
))
fig.show()

In [ ]:
best_log_return_index = dropnaninf(exp).idxmax()
print("best elr:", exp[best_log_return_index])
best_sharpe_index = dropnaninf(sharpe).idxmax()
print("best sharpe:", sharpe[best_sharpe_index])

In [ ]:
exp.fillna(exp.min()*1.1).vbt.volume().show()


